## Federated Learning to Predict Hospital Admissions
### Section 1: Data Prep
Cleans and preps data for ID3 decision tree using federated learning. Utilizing the UCI Diabetes dataset found here: https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

ID3 decision tree requires categorical variables as input.  

In [4]:
import pandas as pd
import numpy as np
import re, os
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import zscore

import warnings
warnings.filterwarnings("ignore")

In [5]:
# [[change to relative paths prior to publishing]]
df_total = pd.read_csv('./0.0_RawData/diabetic_data.csv')

In [6]:
# remove all patient ID variables and variables that I consider not relevant
toRemove = ['encounter_id', 'patient_nbr']
df_total.drop(columns = toRemove, axis = 1, inplace = True)

df_total.columns

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

Split the dataset. In our simulation, we are comparing a small provider participating in a larger "shared data consortium"

In our simulation, the provider is a circulatory system specialty group, so we select all observations that have disease of the circulatory system icd-9 code (approximately 30% of data). Therefore, while the provider dataset is clearly specialized, the aggregate data is general with circulatory patients (based on their second and third diagnoses)

In [7]:
# diseases of the circulatory system (390-459)
heart_codes = ("39", "40", "41", "42", "43", "44", "45")

# sdc = shared data consortium (i.e., aggregate data)
# provider = small provider (this is the only data visible at the time of modeling)
provider = df_total[df_total["diag_1"].str.startswith(heart_codes)]
sdc = df_total.drop(provider.index, axis = 0)

#assert provider.shape[0] + sdc.shape[0] == df_total.shape[0]
provider.shape, sdc.shape

((30391, 48), (71375, 48))

### Clean and prep the data
Various data cleaning techniques such as impute missing values, normalize numeric data, recode variables. For training data only: drop outliers and random undersampling

In [8]:
# identify columns with high missing values
missing_list = []
tot = len(provider)
for column in provider:
    missing = len(provider[provider[column] == "?"])
    missing_list.append((column, missing / tot))
    
missing_list.sort(key = lambda x: x[1], reverse = True)
print(missing_list) # three values of concern: weight, medical_specialty, payer_code

# Missing values
def convertMissing(value):
    if value == "?":
        return "Unknown"
    else:
        return value

# drop columns with high missing and add unknown category for other missing values (all categorical variables)
# medical_specialty is likely captured in diagnoses
# payer_code likely has low predictive power
def handleMissing(df):
    df = df.drop(columns = ['weight', 'payer_code', 'medical_specialty', 'diag_2','diag_3'], axis = 1)
    return df.applymap(convertMissing)
    

provider = handleMissing(provider)
sdc = handleMissing(sdc)

[('weight', 0.9666019545260109), ('medical_specialty', 0.48445263400348787), ('payer_code', 0.44296008686782273), ('race', 0.02250666315685565), ('diag_3', 0.0037840150044421046), ('diag_2', 0.000526471652791945), ('gender', 0.0), ('age', 0.0), ('admission_type_id', 0.0), ('discharge_disposition_id', 0.0), ('admission_source_id', 0.0), ('time_in_hospital', 0.0), ('num_lab_procedures', 0.0), ('num_procedures', 0.0), ('num_medications', 0.0), ('number_outpatient', 0.0), ('number_emergency', 0.0), ('number_inpatient', 0.0), ('diag_1', 0.0), ('number_diagnoses', 0.0), ('max_glu_serum', 0.0), ('A1Cresult', 0.0), ('metformin', 0.0), ('repaglinide', 0.0), ('nateglinide', 0.0), ('chlorpropamide', 0.0), ('glimepiride', 0.0), ('acetohexamide', 0.0), ('glipizide', 0.0), ('glyburide', 0.0), ('tolbutamide', 0.0), ('pioglitazone', 0.0), ('rosiglitazone', 0.0), ('acarbose', 0.0), ('miglitol', 0.0), ('troglitazone', 0.0), ('tolazamide', 0.0), ('examide', 0.0), ('citoglipton', 0.0), ('insulin', 0.0), (

In [9]:
# normalize numeric data 
# using a very simple rule that if abs(skew) > 1 in either dataset then we will normalize
toRemove = ["admission_type_id", "discharge_disposition_id", "admission_source_id"] # actually categorical
p_skew = provider.skew(axis = 0)
p_skew = p_skew[abs(p_skew) > 1]

s_skew = sdc.skew(axis = 0)
s_skew = s_skew[abs(s_skew) > 1]

skewed_vars = list(set(list(p_skew.index) + list(s_skew.index)))
skewed_vars = [var for var in skewed_vars if var not in toRemove]
print("Normalizing with Log(1+x):",skewed_vars)

for var in skewed_vars:
    sdc[var] = np.log1p(sdc[var])
    provider[var] = np.log1p(provider[var]) 
    

Normalizing with Log(1+x): ['number_emergency', 'num_medications', 'num_procedures', 'number_inpatient', 'number_outpatient', 'time_in_hospital']


Recode variables: age and diagnosis codes

In [10]:
# icd 9 code groups from: https://en.wikipedia.org/wiki/List_of_ICD-9_codes
def formatRange(start, end):
    ls = list(range(start, end+1))
    ls = [str(val).zfill(2) for val in ls]
    return re.compile("|".join("^"+val+".*$" for val in ls)) # Enables replacement based on startswith
    
recode_icd = {formatRange(0,13): "Infectious diseases",
             formatRange(14,23): "Neoplasms",
             formatRange(24,27): "Endocrine & Immunity",
             "^28.*$": "Blood diseases",
             formatRange(29,31): "Mental disorders",
             formatRange(32,38): "Nervous System",
             formatRange(39,45): "Circulatory System",
             formatRange(46,51): "Respiratory System",
             formatRange(52,57):"Digestive System",
             formatRange(58,62): "Genitourinary System",
             formatRange(63,67): "Pregnancy & Childbirth",
             formatRange(68,70): "Skin diseases",
             formatRange(71,73): "Musculoskeletal",
             formatRange(74,75): "Congenital Anomalies",
             formatRange(76,77): "Perinatal conditions",
             formatRange(78,79): "Ill-defined conditions",
             formatRange(80,100): "Injury",
             "^E.*$|^V.*$": "External Injuries"
             }

# recode relevant variables
def recodeVars(df):
    for i in range(1,2):
        df['diag_'+str(i)] = df['diag_'+str(i)].str.zfill(2)
        df = df.replace({"diag_"+str(i):recode_icd}, regex = True)
    return df

sdc = recodeVars(sdc)
provider = recodeVars(provider)    

### Split Train and Test
Then finalize data prep: remove outliers and random undersampling

In [11]:
provider_train, provider_test = train_test_split(provider, test_size = .5)
provider_train.readmitted.value_counts(), provider_test.readmitted.value_counts()

(NO     8060
 >30    5382
 <30    1753
 Name: readmitted, dtype: int64, NO     8017
 >30    5453
 <30    1726
 Name: readmitted, dtype: int64)

In [12]:
# removing outliers based on the 4th standard deviation
sdc = sdc[(np.abs(zscore(sdc[skewed_vars])) < 4).all(axis=1)]
provider_train = provider_train[(np.abs(zscore(provider_train[skewed_vars])) <4).all(axis = 1)]

The Id3 algorithm expects categorical variables. We will now bin our continuous variables.

for simplicity sake, we will limit our number of bins to 5

In [13]:
import warnings
warnings.filterwarnings("ignore")

provider_train.columns
cont_vars = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 
             'num_medications','number_outpatient', 'number_emergency', 
             'number_inpatient', 'number_diagnoses']

total_train = pd.concat([provider_train, sdc, provider_test], axis = 0)
# TODO: Remove provider test at final run 

for var in cont_vars:
    # create bins and apply to all datasets
    series, bins = pd.cut(total_train[var], 5, retbins = True)
    bins[0] = float("-inf")
    bins[-1] = float("inf")
    
    provider_train.loc[:,var] = pd.cut(provider_train[var], bins = bins).astype(str)
    provider_test.loc[:,var] = pd.cut(provider_test[var], bins = bins).astype(str)
    sdc.loc[:,var] = pd.cut(sdc[var], bins = bins).astype(str)
    
    print(provider_test[var].unique())
    assert len(provider_test[var].unique()) <= 5

['(1.096, 1.499]' '(1.902, 2.305]' '(1.499, 1.902]' '(2.305, inf]'
 '(-inf, 1.096]']
['(53.4, 79.6]' '(-inf, 27.2]' '(27.2, 53.4]' '(79.6, 105.8]'
 '(105.8, inf]']
['(-inf, 0.389]' '(1.557, inf]' '(1.168, 1.557]' '(0.778, 1.168]'
 '(0.389, 0.778]']
['(2.179, 2.921]' '(2.921, 3.664]' '(-inf, 1.436]' '(1.436, 2.179]'
 '(3.664, inf]']
['(-inf, 0.717]' '(0.717, 1.433]' '(1.433, 2.15]' '(2.867, inf]'
 '(2.15, 2.867]']
['(-inf, 0.832]' '(0.832, 1.664]' '(1.664, 2.495]' '(2.495, 3.327]'
 '(3.327, inf]']
['(-inf, 0.528]' '(0.528, 1.056]' '(1.056, 1.583]' '(1.583, 2.111]'
 '(2.111, inf]']
['(7.0, 10.0]' '(-inf, 4.0]' '(4.0, 7.0]' '(10.0, 13.0]' '(13.0, inf]']


Decision trees usually require balanced datasets. We will use random undersampling on our training sets

In [30]:
# Random Undersampling of both datasets
def underSample(df):
    sampler = RandomUnderSampler()
    fit_sample = sampler.fit_sample(df.drop(columns = 'readmitted'), df['readmitted'])
    cols = df.columns.to_list()
    cols.remove('readmitted')
    cols += ['readmitted']
    return pd.DataFrame(data=np.column_stack(fit_sample),columns=cols)

sdc = underSample(sdc)
provider_train = underSample(provider_train)
sdc.shape, sdc.readmitted.value_counts(), provider_train.shape, provider_train.readmitted.value_counts()

((22575, 43), NO     7525
 <30    7525
 >30    7525
 Name: readmitted, dtype: int64, (5007, 43), <30    1669
 >30    1669
 NO     1669
 Name: readmitted, dtype: int64)

In [31]:
display(sdc.head()) 
display(provider_train.head())

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,AfricanAmerican,Male,[60-70),2,1,4,"(1.902, 2.305]","(53.4, 79.6]","(-inf, 0.389]","(2.179, 2.921]",...,No,Steady,No,No,No,No,No,Ch,Yes,<30
1,AfricanAmerican,Male,[50-60),2,1,2,"(1.499, 1.902]","(27.2, 53.4]","(0.389, 0.778]","(2.179, 2.921]",...,No,No,No,No,No,No,No,No,No,<30
2,Caucasian,Female,[70-80),1,6,7,"(1.902, 2.305]","(53.4, 79.6]","(0.778, 1.168]","(2.921, 3.664]",...,No,No,No,No,No,No,No,No,No,<30
3,Caucasian,Female,[60-70),3,3,1,"(1.096, 1.499]","(53.4, 79.6]","(0.389, 0.778]","(2.921, 3.664]",...,No,Up,No,No,No,No,No,Ch,Yes,<30
4,Caucasian,Male,[80-90),1,1,7,"(-inf, 1.096]","(27.2, 53.4]","(-inf, 0.389]","(2.179, 2.921]",...,No,Steady,No,No,No,No,No,No,Yes,<30


,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Hispanic,Female,[70-80),3,5,1,"(1.096, 1.499]","(53.4, 79.6]","(-inf, 0.389]","(2.179, 2.921]",...,No,Steady,No,No,No,No,No,No,Yes,<30
1,Caucasian,Female,[70-80),5,6,17,"(1.096, 1.499]","(-inf, 27.2]","(-inf, 0.389]","(2.921, 3.664]",...,No,Up,No,No,No,No,No,Ch,Yes,<30
2,Caucasian,Male,[60-70),1,1,7,"(1.096, 1.499]","(-inf, 27.2]","(1.168, 1.557]","(2.179, 2.921]",...,No,No,No,No,No,No,No,No,Yes,<30
3,AfricanAmerican,Male,[70-80),1,3,7,"(1.902, 2.305]","(27.2, 53.4]","(0.389, 0.778]","(2.921, 3.664]",...,No,Up,No,No,No,No,No,Ch,Yes,<30
4,Caucasian,Female,[70-80),3,1,1,"(-inf, 1.096]","(-inf, 27.2]","(1.557, inf]","(2.179, 2.921]",...,No,Down,No,No,No,No,No,Ch,Yes,<30


In [32]:
# Federated learning framework expects label column to be named class
def prep_y(df):
    df = df.rename(columns = {'readmitted':'class'})
    #df['class'] = df['class'].map({'<30':1, '>30':0, 'NO':0})
    return df

provider_train = prep_y(provider_train)
provider_test = prep_y(provider_test)
sdc = prep_y(sdc)

In [33]:
# export data
try:
    os.mkdir(f'{basePath}Aggregator')
except:
    pass

try:
    os.mkdir(f'{basePath}Party1')
except:
    pass

try:
    os.mkdir(f'{basePath}Party2')
except:
    pass

try:
    os.mkdir(f'{basePath}Party3')
except: 
    pass


# randomly split sdc into two parties
np.random.seed(0)
idx = np.random.choice(len(sdc), replace = False, size = int(len(sdc)/2))
sdc_party1 = sdc.iloc[idx]
sdc_party2 = sdc.iloc[~idx]

sdc_party1.to_csv(f'{basePath}Party1/SDC_Party1.csv', index = None)
sdc_party2.to_csv(f'{basePath}Party2/SDC_Party2.csv', index = None)

provider_train.to_csv(f'{basePath}Party3/Provider_Train.csv', index = None)
provider_test.to_csv(f'{basePath}Party3/Provider_Test.csv', index = None)

"""
Save file with potential values for all features to aggregator dataset - so that it can issue complete commands
This is meant to replicate the agreed upon file format by all parties
No actual data is stored, just feature names and potential values
To ensure same length columns, we repeat the last value until max unique value is reached
"""
def feature_values(dfs = []):
    global total_df
    total_df = pd.concat(dfs, axis = 0)
    
    nrow = max(list(map(lambda x: len(total_df[x].unique()), total_df.columns)))
    unique_values =[]
    for var in total_df.columns:
        unique_values.append(list(total_df[var].unique()))

    df = list(map(lambda x: x + [x[-1]]*(nrow-len(x)), unique_values))
    df = pd.DataFrame(df).transpose()
    df.columns = total_df.columns
    return df

Config = feature_values(dfs = [provider_train, sdc_party1, sdc_party2])
Config.to_csv(f'{basePath}Aggregator/Config_DF.csv', index = None)